# GARCH Model

In [5]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.optimize import minimize

In [7]:
class IGGARCH:
    def __init__(self, params=None):
        """
        Initialize the IG-GARCH model with parameters based on 
        equations (10)-(15) from Augustyniak et al. (2021).
        
        Parameters:
        params : dictionary containing the model parameters
            - lambda: equity risk premium (eq. 10)
            - eta: skewness parameter (eq. 10)
            - sigma2: unconditional variance (eq. 13)
            - rho_s: persistence of short-run component (eq. 12)
            - rho_q: persistence of long-run component (eq. 13)
            - c_s: leverage effect parameter for short-run component (eq. 14)
            - c_q: leverage effect parameter for long-run component (eq. 15)
            - a_s: innovation parameter for short-run component (eq. 14)
            - a_q: innovation parameter for long-run component (eq. 15)
        """
        self.params = params if params is not None else {}
        self.h = None  # conditional variance (eq. 11)
        self.s = None  # short-run component (eq. 12)
        self.q = None  # long-run component (eq. 13)

    def initialize_state(self, returns):
        """
        Initialize state variables (h, s, q) based on return data.
        This sets up the initial values for equations (11)-(13) from Augustyniak et al. (2021).
        """
        n = len(returns)
        
        # Initialize with unconditional variance
        sigma2 = self.params.get('sigma2', np.var(returns))
        self.h = np.zeros(n + 1)
        self.s = np.zeros(n + 1)
        self.q = np.zeros(n + 1)
        
        # Set initial values
        self.h[0] = sigma2
        self.s[0] = 0  # Short-run component starts at 0
        self.q[0] = sigma2  # Long-run component starts at unconditional variance (eq. 13)
    
    def initialize_state(self, returns):
        """
        Initialize state variables (h, s, q) based on return data.
        This sets up the initial values for equations (11)-(13) from Augustyniak et al. (2021).
        """
        n = len(returns)
        
        # Initialize with unconditional variance
        sigma2 = self.params.get('sigma2', np.var(returns))
        self.h = np.zeros(n + 1)
        self.s = np.zeros(n + 1)
        self.q = np.zeros(n + 1)
        
        # Set initial values
        self.h[0] = sigma2
        self.s[0] = 0  # Short-run component starts at 0
        self.q[0] = sigma2  # Long-run component starts at unconditional variance (eq. 13)

    def simulate(self, returns, r=0.0):
        """
        Simulate the IG-GARCH model using given returns and parameters.
        Implements equations (10)-(15) from Augustyniak et al. (2021).
        
        Parameters:
        returns : array-like, shape (n_samples,)
            The asset returns
        r : float
            Risk-free rate
        """
        n = len(returns)
        
        # Extract parameters
        lam = self.params['lambda']
        eta = self.params['eta']
        sigma2 = self.params['sigma2']
        rho_s = self.params['rho_s']
        rho_q = self.params['rho_q']
        c_s = self.params['c_s']
        c_q = self.params['c_q']
        a_s = self.params['a_s']
        a_q = self.params['a_q']
        
        # Initialize state variables if they haven't been already
        if self.h is None or len(self.h) <= n:
            self.initialize_state(returns)
        
        for t in range(n):
            h_t = self.h[t]
            s_t = self.s[t]
            q_t = self.q[t]
            
            # Mean equation (eq. 10)
            y_t = returns[t]
            mean_t = r + lam * h_t
            
            # Compute normalized residual based on IG distribution (eq. 10)
            z_t = (y_t - mean_t) / eta
            
            # Compute innovation terms (eq. 14-15)
            zeta_s = c_s * z_t + a_s * h_t**2 / z_t - c_s * h_t / eta**2 - a_s * eta**2 * h_t - a_s * eta**4
            zeta_q = c_q * z_t + a_q * h_t**2 / z_t - c_q * h_t / eta**2 - a_q * eta**2 * h_t - a_q * eta**4
            
            # Update state variables (eq. 12-13)
            self.s[t+1] = rho_s * s_t + zeta_s
            self.q[t+1] = sigma2 + rho_q * (q_t - sigma2) + zeta_q
            
            # Update total variance (eq. 11)
            self.h[t+1] = self.s[t+1] + self.q[t+1]
            
        return self.h[1:n+1]  # Return conditional variances
    
    def fit(self, returns, r=0.0):
        """
        Fit the IG-GARCH model to return data using maximum likelihood.
        Based on the likelihood of the IG distribution in equation (10) from Augustyniak et al. (2021).
        
        Parameters:
        returns : array-like, shape (n_samples,)
            The asset returns
        r : float
            Risk-free rate
        """
        # Define the negative log-likelihood function to minimize
        def neg_log_likelihood(params):
            lam = params[0]
            eta = params[1]
            sigma2 = params[2]
            rho_s = params[3]
            rho_q = params[4]
            c_s = params[5]
            c_q = params[6]
            a_s = params[7]
            a_q = params[8]
            
            # Update parameters dictionary
            self.params = {
                'lambda': lam,
                'eta': eta,
                'sigma2': sigma2,
                'rho_s': rho_s,
                'rho_q': rho_q,
                'c_s': c_s,
                'c_q': c_q,
                'a_s': a_s,
                'a_q': a_q
            }
            
            # Initialize state variables
            self.initialize_state(returns)
            
            n = len(returns)
            log_likelihood = 0.0
            
            for t in range(n):
                # Calculate conditional variance components
                h_t = self.h[t]
                s_t = self.s[t]
                q_t = self.q[t]
                
                # Compute the log-likelihood contribution for this observation
                y_t = returns[t]
                mean_t = r + lam * h_t
                
                # Extract the error term
                error_term = y_t - mean_t  # This is η z_t in the paper

                # Calculate z_t (IG-distributed random variable)
                z_t = error_term / eta  # This gives us the IG random variable

                # Check if z_t is positive (IG distribution domain)
                if z_t <= 0:  # IG distribution is defined for positive values
                    raise ValueError(f"Negative z_t value detected ({z_t}). IG distribution is only defined for positive values.")


                # IG distribution log-likelihood
                # For IG(μ,λ) where μ = h_{t-1}/η² and λ = (h_{t-1})²/η⁴
                # since for IG disttribution, the variance is equal to μ³/λ
                mu = h_t / eta**2
                lambda_param = h_t**2 / eta**4

                # by solving log of CDF for IG(μ,λ) we get the following log-likelihood
                log_likelihood += 0.5 * np.log(lambda_param) - 0.5 * np.log(2 * np.pi) - 1.5 * np.log(z_t) - \
                                (lambda_param * (z_t - mu)**2) / (2 * mu**2 * z_t)

                # Compute innovation terms (Eq. 14-15)
                # Note: z_t is now correctly defined
                zeta_s = c_s * z_t + a_s * h_t**2 / z_t - c_s * h_t / eta**2 - a_s * eta**2 * h_t - a_s * eta**4
                zeta_q = c_q * z_t + a_q * h_t**2 / z_t - c_q * h_t / eta**2 - a_q * eta**2 * h_t - a_q * eta**4

                # Update state variables (Eq. 12-13)
                self.s[t+1] = rho_s * s_t + zeta_s
                self.q[t+1] = sigma2 + rho_q * (q_t - sigma2) + zeta_q

                # Update total variance (Eq. 11)
                self.h[t+1] = self.s[t+1] + self.q[t+1]
            
            return -log_likelihood
        
        # Initial parameter guess based on Table 2 in Augustyniak et al. (2021)
        initial_params = [
            self.params.get('lambda', 2.5),
            self.params.get('eta', -0.0006),
            self.params.get('sigma2', 1.2e-4),
            self.params.get('rho_s', 0.8),
            self.params.get('rho_q', 0.99),
            self.params.get('c_s', 1.7e-6),
            self.params.get('c_q', 2.5e-6),
            self.params.get('a_s', 2.2e7),
            self.params.get('a_q', 3.6e7)
        ]
        
        # Parameter constraints based on Table 2 in Augustyniak et al. (2021)
        bounds = [
            (-10, 10),           # lambda
            (-0.001, -0.0001),   # eta (adjusted to match paper's values)
            (1e-6, 1e-3),        # sigma2
            (0.5, 0.999),        # rho_s
            (0.98, 0.9999),      # rho_q
            (1e-7, 1e-5),        # c_s (adjusted to match paper's values)
            (1e-7, 1e-5),        # c_q (adjusted to match paper's values)
            (1e6, 1e8),          # a_s (adjusted to match paper's values)
            (1e6, 1e8)           # a_q (adjusted to match paper's values)
        ]
        
        # Minimize negative log-likelihood
        result = minimize(neg_log_likelihood, initial_params, bounds=bounds, method='L-BFGS-B')
        
        # Extract and store optimized parameters
        params = result.x
        self.params = {
            'lambda': params[0],
            'eta': params[1],
            'sigma2': params[2],
            'rho_s': params[3],
            'rho_q': params[4],
            'c_s': params[5],
            'c_q': params[6],
            'a_s': params[7],
            'a_q': params[8]
        }
        
        # Re-run to store final state variables
        self.initialize_state(returns)
        self.simulate(returns, r)
        
        return result
    
    def forecast(self, horizon=1):
        """
        Forecast conditional variance h_t for horizon steps ahead.
        Based on the dynamics from equations (11)-(13) from Augustyniak et al. (2021).
        
        Parameters:
        horizon : int
            Number of steps ahead to forecast
            
        Returns:
        array-like, shape (horizon,)
            Forecasted conditional variances
        """
        # Extract parameters
        sigma2 = self.params['sigma2']
        rho_s = self.params['rho_s']
        rho_q = self.params['rho_q']
        
        # Get latest state values
        s_t = self.s[-1]
        q_t = self.q[-1]
        
        # Initialize forecasts
        s_forecasts = np.zeros(horizon)
        q_forecasts = np.zeros(horizon)
        h_forecasts = np.zeros(horizon)
        
        # Generate forecasts
        for h in range(horizon):
            if h == 0:
                s_forecasts[h] = rho_s * s_t
                q_forecasts[h] = sigma2 + rho_q * (q_t - sigma2)
            else:
                s_forecasts[h] = rho_s * s_forecasts[h-1]
                q_forecasts[h] = sigma2 + rho_q * (q_forecasts[h-1] - sigma2)
            
            h_forecasts[h] = s_forecasts[h] + q_forecasts[h]
        
        return h_forecasts